In [13]:
''' Dataset information
| Data File                   | Size     | Comment                                    |
|:----------------------------|:---------|:-------------------------------------------|
| _raw_df_channels.tsv.gz     | 6.4 MB   | See 'create_sample_datasets_tsv.ipynb'     |
| _raw_df_timeseries.tsv.gz   | 653.1 MB | See 'create_sample_datasets_tsv.ipynb'     |
| _raw_yt_metadata.jsonl.zst  | 14.7 GB  | See 'create_sample_datasets_json.ipynb'    |
| df_channels_en.tsv.gz       | 6.0 MB   | See 'create_sample_datasets_tsv.ipynb'     |
| df_timeseries_en.tsv.gz     | 571.1 MB | See 'create_sample_datasets_tsv.ipynb'     |
| num_comments.tsv.gz         | 754.6 MB | See 'create_sample_datasets_tsv.ipynb'     |
| num_comments_authors.tsv.gz | 1.4 GB   | See 'create_sample_datasets_tsv.ipynb'     |
| youtube_comments.tsv.gz     | 77.2 GB  | See 'create_sample_datasets_tsv.ipynb'     |
| yt_metadata_en.jsonl.gz     | 13.6 GB  | See 'create_sample_datasets_json.ipynb'    |
| yt_metadata_helper.feather  | 2.8 GB   | Sample dataset created.                    |
'''

" Dataset information\n| Data File                   | Size     | Comment                                    |\n|:----------------------------|:---------|:-------------------------------------------|\n| _raw_df_channels.tsv.gz     | 6.4 MB   | See 'create_sample_datasets_tsv.ipynb'     |\n| _raw_df_timeseries.tsv.gz   | 653.1 MB | See 'create_sample_datasets_tsv.ipynb'     |\n| _raw_yt_metadata.jsonl.zst  | 14.7 GB  | See 'create_sample_datasets_json.ipynb'    |\n| df_channels_en.tsv.gz       | 6.0 MB   | See 'create_sample_datasets_tsv.ipynb'     |\n| df_timeseries_en.tsv.gz     | 571.1 MB | See 'create_sample_datasets_tsv.ipynb'     |\n| num_comments.tsv.gz         | 754.6 MB | See 'create_sample_datasets_tsv.ipynb'     |\n| num_comments_authors.tsv.gz | 1.4 GB   | See 'create_sample_datasets_tsv.ipynb'     |\n| youtube_comments.tsv.gz     | 77.2 GB  | See 'create_sample_datasets_tsv.ipynb'     |\n| yt_metadata_en.jsonl.gz     | 13.6 GB  | See 'create_sample_datasets_json.ipynb'    |

In [14]:
import os
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.feather as feather

# Verify current working directory
print(f"Current working directory: {os.getcwd()}")

# Ensure Arrow uses system allocator for stability on Windows
os.environ.setdefault("ARROW_DEFAULT_MEMORY_POOL", "system")

Current working directory: d:\_edd\_local_repos\cs513_data_mining\trendy-tube\Preprocessing


'system'

In [15]:
# Read from the feather file via memory-mapped Arrow Table (low memory)
FEATHER_FILE_NAME = 'yt_metadata_helper'
FEATHER_PATH = f'../RawData/{FEATHER_FILE_NAME}.feather'

# Optionally limit columns to reduce memory: columns=[...]
table = feather.read_table(FEATHER_PATH, memory_map=True)
print(f"Feather rows: {table.num_rows:,}, cols: {table.num_columns}")

Feather rows: 72,924,794, cols: 8


In [16]:
# Random sample up to SAMPLE_SIZE rows
SAMPLE_SIZE = 50_000
n = table.num_rows
k = min(SAMPLE_SIZE, n)
rng = np.random.default_rng(42)
idx = np.sort(rng.choice(n, size=k, replace=False))

sample_table= table.take(pa.array(idx, type=pa.int64()))
df_sample = sample_table.to_pandas(use_threads=True)
print(f"Sampled DataFrame shape: {df_sample.shape}")
df_sample.head()

Sampled DataFrame shape: (50000, 8)


,categories,channel_id,dislike_count,display_id,duration,like_count,upload_date,view_count
0,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,1.0,J_WLXskvMQo,1033,8.0,2016-09-15,2551.0
1,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,0.0,evnxtBrBY0s,1979,11.0,2016-02-01,1152.0
2,Gaming,UCzWrhkg9eK5I8Bm3HfV-unA,1.0,Ia4RTAbSk1o,2380,11.0,2017-06-17,6177.0
3,Sports,UCzWn_gTaXyH5Idyo8Raf7_A,63.0,2PXjv5fGHoo,925,3062.0,2017-12-11,142104.0
4,Gaming,UCzWfqsjlvzAOUozsGFR3kbg,22.0,OHXFN_km9wg,42,542.0,2015-04-01,23076.0


In [17]:
# Write to .csv file
df_sample.to_csv(f'../SampleData/{FEATHER_FILE_NAME}' + '_sample.csv', index=False)